In [1]:
from typing import List, Optional
from torch.optim.optimizer import Optimizer

import os
import hydra
from torch import nn
from omegaconf import DictConfig
import pytorch_lightning as pl
from pytorch_lightning import (
    Callback,
    LightningDataModule,
    LightningModule,
    Trainer,
    seed_everything,
)
from pytorch_lightning.loggers import LightningLoggerBase

from torch.utils.data import Dataset
from src.utils import utils
from hydra import compose, initialize
from omegaconf import OmegaConf
from src.metrics.bounding_box import BoundingBox
from src.metrics.enumerators import BBType, CoordinatesType, BBFormat, MethodAveragePrecision
from enum import Enum

# GlobalHydra.instance().clear()
initialize(config_path="pest-monitoring-new/configs/", job_name="test_app")

hydra.initialize()

In [2]:
config = compose(config_name="pest_classification.yaml", overrides=[])

In [3]:
datamodule: LightningDataModule = hydra.utils.instantiate(config.datamodule, data_config = config.datamodule, _recursive_=False)
datamodule.setup()

In [4]:
criterion = hydra.utils.instantiate(config.model.loss)
network = hydra.utils.instantiate(config.model.network)

In [5]:
model = hydra.utils.instantiate(config.model, config.model, _recursive_=False)

/usr/local/lib/python3.6/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


### Trial Run (Check if this works)

In [6]:
for batch_id, batch in enumerate(datamodule.train_dataloader()):
    break

In [7]:
images, targets = batch
images.shape, targets.shape

(torch.Size([32, 3, 224, 224]), torch.Size([32]))

In [8]:
preds = network(images)
preds.shape

torch.Size([32, 2])

In [9]:
import torchmetrics
import torch.nn.functional as F

acc = torchmetrics.Accuracy()
acc(F.softmax(preds, dim=1), targets)

tensor(0.4688)

In [8]:
x, y = datamodule.dataset_train[0]

<class 'torch.Tensor'>
tensor([0.])


In [10]:
int(y.item())

0

In [13]:
criterion(preds, targets)

RuntimeError: 1D target tensor expected, multi-target not supported

### Define Network

In [3]:
model: LightningModule = hydra.utils.instantiate(config.model,  model_config = config.model, _recursive_=False)

/root/.local/lib/python3.6/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


In [7]:
# net(batch[0])